### Medallion-Spark-dbt-project

In [0]:
dbutils.secrets.listScopes()

[SecretScope(name='databricsScope')]

#### Mounting the Gold, Silver and Bronze Storages

In [0]:
storage_account_name = 'medallionsaabdulshakur'
secret_scope = 'databricsScope'
secret_name = 'adlsSecret'

medallion = ['gold','silver','bronze']

#mount the strorages
try:
    for medal in medallion:
        dbutils.fs.mount(
            source=f"wasbs://{medal}@{storage_account_name}.blob.core.windows.net/",
            mount_point=f'/mnt/{medal}',
            extra_configs = {f'fs.azure.account.key.{storage_account_name}.blob.core.windows.net': dbutils.secrets.get(secret_scope, secret_name)}
        )
except:
    pass

#### Create the bronze schema

In [0]:
spark.sql('create database if not exists bronze')


DataFrame[]

#### Dynamically create the table catalogs

In [0]:
fileName = dbutils.widgets.get('fileName')
folderName = dbutils.widgets.get('folderName')
schemaName = dbutils.widgets.get('schemaName')
tableName = f"bronze.{folderName}_{schemaName}_{fileName}"
spark.sql(f"""
         CREATE TABLE IF NOT EXISTS {tableName}
          USING PARQUET LOCATION '/mnt/bronze/{folderName}/{schemaName}/{fileName}.parquet'
          """)